# Initialising The Medical School as an Issuing Authority

The Medical School needs to write a schema for the Primary Medical Qualification (PMQ) credential they intend to issue to all medics that pass their exams. In a realistic scenario, this schema may be defined by a different authority, for example the medical schools boad (or the GMC), who oversee education standards and those trusted to implement it. In that case the Edinburgh Medical School would only need to write a credential definition (a public key) enabling them to issue credentails against this schema.

![Medic Education](./medic-education.png)


### Imports

In [1]:
from aries_cloudcontroller import AriesAgentController
import os
from termcolor import colored

### Initialise the Agent Controller

In [2]:
api_key = os.getenv("ACAPY_ADMIN_API_KEY")
admin_url = os.getenv("ADMIN_URL")

print(f"Initialising a controller with admin api at {admin_url} and an api key of {api_key}")
agent_controller = AriesAgentController(admin_url,api_key)

Initialising a controller with admin api at http://edinburgh-medical-school-agent:3021 and an api key of adminApiKey


## Write DID to the Public Ledger

Note: if defined a ACAPY_WALLET_SEED value for your agent then this function will return a DID, but this DID still needs to be written to the ledger. If you did not define a seed you will need to create a DID first.

In [3]:
public_did_response = await agent_controller.wallet.get_public_did()
print(public_did_response)

{'result': {'did': '3jnXQcj9VLFjcUbtDVZZzV', 'verkey': '2VYWDcUQrLk2EMVBYxdVEHZtzhdACy9JqRBvHtck7U98', 'posture': 'public'}}


In [4]:
if public_did_response["result"]:
    did_obj = public_did_response["result"]
    print("Already have a public did")
else:
    create_did_response = await agent_controller.wallet.create_did()
    did_obj = create_did_response['result']
print("DID", did_obj)

Already have a public did
DID {'did': '3jnXQcj9VLFjcUbtDVZZzV', 'verkey': '2VYWDcUQrLk2EMVBYxdVEHZtzhdACy9JqRBvHtck7U98', 'posture': 'public'}


## Not Needed When using a Local VON Network

In [12]:
# write new DID to Sovrin Stagingnet
# import requests
# import json 

# url = 'https://selfserve.sovrin.org/nym'

# payload = {"network":"stagingnet","did": did_obj["did"],"verkey":did_obj["verkey"],"paymentaddr":""}

# # Adding empty header as parameters are being sent in payload
# headers = {}

# r = requests.post(url, data=json.dumps(payload), headers=headers)
# print(r.json())

{'statusCode': 200, 'headers': {'Access-Control-Allow-Origin': '*'}, 'body': '{"statusCode": 200, "W4t2Pa4XR1qBDvwhVcn4CY": {"status": "Success", "statusCode": 200, "reason": "Successfully wrote NYM identified by W4t2Pa4XR1qBDvwhVcn4CY to the ledger with role ENDORSER"}}'}
200


## Accept Transaction Author Agreement

## Also Not Needed when using Local VON Network

Although the Sovrin StagingNet is permissionless, before DID's have the authority to write to the ledger they must accept something called a transaction author agreement by signing it using the DID they have on the ledger.

As a global public ledger, the Sovrin Ledger and all its participants are subject to privacy and data protection regulations such as the EU General Data Protection Regulation (GDPR). These regulations require that the participants be explicit about responsibilities for Personal Data.

To clarify these responsibilities and provide protection for all parties, the Sovrin Governance Framework Working Group developed an agreement between Transaction Authors and the Sovrin Foundation. The TAA can be found at Sovrin.org. It ensures that users are aware of and consent to the fact that all data written to the Sovrin Ledger cannot be removed, even if the original author of the transaction requests its removal.

The TAA outlines the policies that users must follow when interacting with the Sovrin Ledger. When a user’s client software is preparing a transaction for submission to the network, it must include a demonstration that the user had the opportunity to review the current TAA and accept it. This is done by including some additional fields in the ledger write transaction: 

* A hash of the agreement
* A date when the agreement was accepted, and
* A string indicating the user interaction that was followed to obtain the acceptance.

The Indy client API used by Sovrin has been extended to allow users to review current and past agreements and to indicate acceptance through an approved user interaction pattern. - source: https://sovrin.org/preparing-for-the-sovrin-transaction-author-agreement/

For more details on TAA please read more at the following links:
* [Preparing for the Sovrin Transaction Author Agreement](https://sovrin.org/preparing-for-the-sovrin-transaction-author-agreement/)
* [How the recent approval of the Sovrin Governance Framework v2 affects Transaction Authors
](https://sovrin.org/how-the-recent-approval-of-the-sovrin-governance-framework-v2-affects-transaction-authors/)
* [TAA v2](https://github.com/sovrin-foundation/sovrin/blob/master/TAA/TAA.md)
* [TAA Acceptance Mechanism List (AML)](https://github.com/sovrin-foundation/sovrin/blob/master/TAA/AML.md)

In [5]:
# taa_response = await agent_controller.ledger.get_taa()
# TAA = taa_response['result']['taa_record']
# TAA['mechanism'] = "service_agreement"
# await agent_controller.ledger.accept_taa(TAA)

## Assign Agent Public DID if Not Set

Will only be ran if ACAPY_WALLET_SEED not initially set.

In [5]:
if did_obj["posture"] != "public":
    response = await agent_controller.wallet.assign_public_did(did_obj["did"])
print("Successfully intialised agent with Public DID : ", did_obj["did"])

Successfully intialised agent with Public DID :  3jnXQcj9VLFjcUbtDVZZzV


## Writing Schema

Write as many schema to the ledger as you like. Be sure to store each schema_id, you will need these when it comes to authoring credential defintition transactions and issuing credentials against this schema.

Uncomment and copy the below cell as many times as you need. Be sure to update any arugments surrounded by <> with your own details.

In [6]:
# Define you schema name - must be unique on the ledger
schema_name = "Primary Medical Qualification"
# Can version the schema if you wish to update it
schema_version = "0.0.1"
# Define any list of attributes you wish to include in your schema
attributes = ["Name", "University", "Date Issued"]

response = await agent_controller.schema.write_schema(schema_name, attributes, schema_version)
pmq_schema = response["schema_id"]

## Using External Schema

If a Primary Medical Qualification schema was defined and managed by another entity, then the Edinburgh Medical School would only have to identify the schema through it's identifier rather than authoring the schema themselves

In [8]:
# <EXTERNAL SCHEMA ID> = "<SOME SCHEMA ID>"

## Writing Credential Definitions

For all schema you intend to issue credentials against your agent must author a credential definition transaction to the public ledger. This specifies the public cryptographic material your agent will use to sign all credentials issued against a specific schema. 

Again uncomment and copy this cell as often as you need. Remebering to update the arguments in <> to specify your schema identifiers. Store each credential definition identifier in a unique variable.

In [7]:
# Tag and group specific credential definitions
tag = "default"

# Make Cred Def support revocation. Credentials issued using this definition will be able to be revoked.
support_revocation = False

cred_def_response = await agent_controller.definitions.write_cred_def(pmq_schema, tag, support_revocation)
pmq_cred_def_id = cred_def_response["credential_definition_id"]

## Persist Identifiers for use throughout other business logic notebooks associated with this agent

The schema_id and cred_def_id value pairs are required whenever issuing credentials, and also can be used to constrain acceptable proof requests. In a real application these values might be stored in environment variables or most likely in a database. For notebooks we have found it easier to store as string values in a cell and then load these values into the jupyter store so that they can be fetched across multiple notebooks.

As such you are recommended to print out each of the schema and cred def identifiers used by your agent and copy them across to your **main** business logic notebook where you should store them in a variable and save them to the jupyter store. Remember, you should only be running this notebook once so having this logic in here will not be useful.


In [8]:
print(f"pmq_schema_id = '{pmq_schema}'")
print(f"pmq_cred_def_id ='{pmq_cred_def_id}'")

pmq_schema_id = '3jnXQcj9VLFjcUbtDVZZzV:2:Primary Medical Qualification:0.0.1'
pmq_cred_def_id ='3jnXQcj9VLFjcUbtDVZZzV:3:CL:10:default'


## Terminate Controller


In [9]:
await agent_controller.terminate()